In [21]:
import nltk
import string

In [2]:
import pandas as pd
import zipfile

In [3]:
zf=zipfile.ZipFile('spam_ham_dataset.csv.zip')
mf=zipfile.ZipFile('emails.csv.zip')

In [4]:
df=pd.read_csv(zf.open('spam_ham_dataset.csv'))
af=pd.read_csv(mf.open('emails.csv'))

In [6]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [8]:
af.head(10)

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0
5,Email 6,4,5,1,4,2,3,45,1,0,...,0,0,0,0,0,0,0,0,0,1
6,Email 7,5,3,1,3,2,1,37,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Email 8,0,2,2,3,1,2,21,6,0,...,0,0,0,0,0,0,0,1,0,1
8,Email 9,2,2,3,0,0,1,18,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Email 10,4,4,35,0,1,0,49,1,16,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from nltk.corpus import stopwords

In [27]:
import string

In [28]:
df.shape

(5171, 4)

In [29]:
df.columns

Index(['Unnamed: 0', 'label', 'text', 'spam'], dtype='object')

In [30]:
df=df.drop('Unnamed: 0',1)

In [31]:
df.head(5)

,label,text,spam
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [32]:
df.shape

(5171, 3)

In [33]:
df=df.rename(columns={"label_num":"spam"})

In [34]:
df.head(5)

,label,text,spam
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [35]:
df.shape

(5171, 3)

In [36]:
df.drop_duplicates(inplace=True)

In [37]:
df.shape

(4993, 3)

In [38]:
df.isnull().sum()

label    0
text     0
spam     0
dtype: int64

In [39]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [40]:
def process_text(text):
    
    nopunc=[char for char in text if char not in string.punctuation]
    nopunc=''.join(nopunc)
    clean_words=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [41]:
df['text'].head(5).apply(process_text)

0    [Subject, enron, methanol, meter, 988291, foll...
1    [Subject, hpl, nom, january, 9, 2001, see, att...
2    [Subject, neon, retreat, ho, ho, ho, around, w...
3    [Subject, photoshop, windows, office, cheap, m...
4    [Subject, indian, springs, deal, book, teco, p...
Name: text, dtype: object

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
messages_bow=CountVectorizer(analyzer=process_text).fit_transform(df['text'])

In [43]:
messages_bow

<4993x50381 sparse matrix of type '<class 'numpy.int64'>'
	with 367956 stored elements in Compressed Sparse Row format>

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(messages_bow,df['spam'],test_size=0.20,random_state=0)

In [45]:
messages_bow.shape

(4993, 50381)

In [57]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train,y_train)

In [34]:
print(classifier.predict(X_train))

[1 0 1 ... 0 0 1]


In [35]:
print(y_train.values)

[1 0 1 ... 0 0 1]


In [39]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(X_train)
print(classification_report(y_train,pred))
print()
print('Confusion Matrix: \n ', confusion_matrix(y_train,pred))
print()
print('Accuracy',accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      2809
           1       0.98      0.99      0.99      1185

    accuracy                           0.99      3994
   macro avg       0.99      0.99      0.99      3994
weighted avg       0.99      0.99      0.99      3994


Confusion Matrix: 
  [[2787   22]
 [  13 1172]]

Accuracy 0.9912368552829244


In [40]:
print(classifier.predict(X_test))
print(y_test.values)

[0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 1 1 0 1 1
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 1 0 0
 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1
 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 

In [41]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(X_test)
print(classification_report(y_test,pred))
print()
print('Confusion Matrix: \n ', confusion_matrix(y_test,pred))
print()
print('Accuracy',accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       722
           1       0.95      0.96      0.96       277

    accuracy                           0.98       999
   macro avg       0.97      0.97      0.97       999
weighted avg       0.98      0.98      0.98       999


Confusion Matrix: 
  [[709  13]
 [ 11 266]]

Accuracy 0.975975975975976


In [46]:
af.shape

(5172, 3002)

In [47]:
L=af.iloc[:,-1].values

In [48]:
L

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [53]:
J=af.iloc[:,1:3001]

In [54]:
J

,the,to,ect,and,for,of,a,you,hou,in,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,0,0,1,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,13,24,6,6,2,102,1,27,18,...,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,8,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,0,5,22,0,5,1,51,2,10,1,...,0,0,0,0,0,0,0,0,0,0
4,7,6,17,1,5,2,57,0,9,3,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,2,2,2,3,0,0,32,0,0,5,...,0,0,0,0,0,0,0,0,0,0
5168,35,27,11,2,6,5,151,4,3,23,...,0,0,0,0,0,0,0,0,1,0
5169,0,0,1,1,0,0,11,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5170,2,7,1,0,2,1,28,2,0,8,...,0,0,0,0,0,0,0,0,1,0


In [55]:
j_train,j_test,l_train,l_test = train_test_split(J,L,test_size = 0.20)

In [58]:
classifier = MultinomialNB().fit(j_train,l_train)

In [59]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(j_test)
print(classification_report(l_test,pred))
print()
print('Confusion Matrix: \n ', confusion_matrix(l_test,pred))
print()
print('Accuracy',accuracy_score(l_test,pred))

              precision    recall  f1-score   support

           0       0.97      0.94      0.95       739
           1       0.85      0.92      0.88       296

    accuracy                           0.93      1035
   macro avg       0.91      0.93      0.92      1035
weighted avg       0.93      0.93      0.93      1035


Confusion Matrix: 
  [[691  48]
 [ 25 271]]

Accuracy 0.9294685990338164
